In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go # data viz

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
raw_data = pd.read_csv("/kaggle/input/coronavirus-data-set-free-api-for-realtime-data/coronavirus.csv", header=0)
raw_data.head()

In [ ]:
transformed_data_1 = raw_data[raw_data.columns[8: 62]].fillna(0)
transformed_data_1 = transformed_data_1.astype(int)
transformed_data_1["CountryOrRegion"] = raw_data["CountryOrRegion"]
transformed_data_1["Type"] = raw_data["Type"]
transformed_data_2 = transformed_data_1.groupby(["CountryOrRegion", "Type"]).sum()
transformed_data_2.reset_index(inplace=True) 
transformed_data_2

In [ ]:
transformed_data_2 = transformed_data_2.loc[transformed_data_2["3/15/2020"] >= 100]

confirmed_cases = transformed_data_2.loc[transformed_data_2["Type"] == "Confirmed"]
confirmed_cases.set_index("CountryOrRegion", inplace=True)
confirmed_cases.drop(columns="Type", inplace=True)

deceased = transformed_data_2.loc[transformed_data_2["Type"] == "Deaths"]
deceased.set_index("CountryOrRegion", inplace=True)
deceased.drop(columns="Type", inplace=True)

recovered = transformed_data_2.loc[transformed_data_2["Type"] == "Recovered"]
recovered.set_index("CountryOrRegion", inplace=True)
recovered.drop(columns="Type", inplace=True)

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=confirmed_cases.index,
        y=confirmed_cases["3/15/2020"],
        name="Confirmed",
        marker_color='rgb(55, 83, 109)'
    )
)
fig.add_trace(
    go.Bar(
        x=deceased.index,
        y=deceased["3/15/2020"],
        name="Deaths",
        marker_color='rgb(26, 118, 255)'
    )
)
fig.add_trace(
    go.Bar(
        x=recovered.index,
        y=recovered["3/15/2020"],
        name="Recovered",
        marker_color='rgb(255, 105, 180)'
    )
)
fig.update_layout(title="Count of confirmed cases the 15th of March 2020 (>= 100 cases)",
                 template="seaborn")
fig.show()

In [ ]:
case_fatality = deceased["3/15/2020"] / (deceased["3/15/2020"] + recovered["3/15/2020"]) * 100
case_fatality.fillna(0, inplace=True)
case_fatality

In [ ]:
fig = go.Figure()
for country in transformed_data_2["CountryOrRegion"].drop_duplicates().tolist():
    fig.add_trace(
        go.Scatter(
            x=confirmed_cases.columns,
            y=confirmed_cases[confirmed_cases.index == country].values[0],
            mode="lines+markers",
            name=country
        )
    )
fig.update_layout(title="Count of confirmed case (cases > 100)")
fig.show()


In [ ]:
fig = go.Figure()
for country in confirmed_cases.index:
    confirmed_cases_for_current_country = confirmed_cases[confirmed_cases.index == country].values[0]
    confirmed_cases_for_current_country = [date for date in confirmed_cases_for_current_country if date >= 100]
    fig.add_trace(
        go.Scatter(
            x=np.arange(1, len(confirmed_cases_for_current_country)+1),
            y=confirmed_cases_for_current_country,
            mode="lines+markers",
            #hoverinfo=""
            name=country
        )
    )
for r in [2, 3, 5, 7]:
    trace=[100 * 2**(i/r) for i in np.arange(1, 54)]
    fig.add_trace(
        go.Scatter(
            x=np.arange(1, 54),
            y=trace,
            mode="lines",
            line={"dash": "dash", "color":"MediumPurple", "width":2}
        )
    )
    text_x=np.argmax(np.array(trace)>=90000) if trace[-1] >= 90000 else 50
    text_y=80000 if trace[-1] >= 90000 else trace[-1]
    fig.add_trace(
        go.Scatter(
            x=[text_x],
            y=[text_y],
            text=[f"Double every {r} days"],
            mode="text",
        )
    )
fig.update_layout(
    title="Flattening the curve: day one since the 100th case",
    yaxis={"range":(-1000, 90000)}
)
fig.show()

In [ ]:
confirmed_cases_for_current_country = transformed_data_2.loc[transformed_data_2["CountryOrRegion"] == "France"]
confirmed_cases_for_current_country